## Run the pipeline:

In [ ]:
from core.meta import MetaInterface

In [ ]:
d = MetaInterface(
    project_config_filepath="test_data/project_config.yaml",
    project_name="test_data",
)

In [ ]:
d.add_recording_config("test_data/Server_structure/Calibrations/220922/recording_config_220922.yaml")

In [ ]:
d.initialize_meta_config()
d.remove_recordings()

In [ ]:
d.create_recordings(recreate_undistorted_plots=True)

In [ ]:
d.synchronize_recordings(verbose=True)

In [ ]:
d.create_calibrations(
    ground_truth_config_filepath="test_data/ground_truth_config.yaml", recreate_undistorted_plots=True
)

In [ ]:
d.synchronize_calibrations(verbose=True)

In [ ]:
d.exclude_markers(all_markers_to_exclude_config_path = "test_data/markers_to_exclude_config.yaml", verbose=False)

In [ ]:
d.calibrate(verbose=1, calibrate_optimal=True)

In [ ]:
d.triangulate_recordings()

In [ ]:
d.normalize_recordings(normalization_config_path="test_data/normalization_config.yaml", verbose=True)

In [ ]:
d.add_triangulated_csv_to_database("test.csv")

In [ ]:
# create empty database
import pandas as pd

df = pd.DataFrame(
    {},
    columns=[
        "recording",
        "date",
        "session_id",
        "paradigm",
        "subject_id",
        "group_id",
        "batch",
        "trial_id",
    ],
)
df.to_csv("test.csv", index=False)

### The meta .yaml file contains all information stored in d.meta

In [ ]:
d.export_meta_to_yaml(filepath="test_data/test_data.yaml")

#### To inspect the files, we recommend https://www.treedoc.org/, which creates a tree-like structured view for .yaml files

In [ ]:
d.objects["triangulation_recordings_objects"][
    "206_F2-63_220922_OTE"
].create_triangulated_video("test_video", "test_data/video_plotting_config_OTE.yaml")

In [ ]:
#d.objects['triangulation_recordings_objects']['206_F2-63_220922_OTE'].metadata_from_videos['Bottom'].fps